In [ ]:
from ultralytics import YOLO
import cv2
import plotly.graph_objects as go
import numpy as np
import threading
import time
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import json

# Load the trained model
model = YOLO("Training/runs/detect/train18/weights/best.pt")

# Open the laptop webcam
cap = cv2.VideoCapture(0)  

# Object detection count
object_counts = {"pizza": 0, "burger": 0, "fries": 0}

# start the real time graph visuaization
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Object Detection Count"),
    dcc.Graph(id="live-bar-chart"),
    dcc.Interval(id="interval-update", interval=500, n_intervals=0) 
])

# Function to update bar chart
@app.callback(
    Output("live-bar-chart", "figure"),
    [Input("interval-update", "n_intervals")]
)
def update_bar_chart(n):
    global object_counts

    fig = go.Figure(data=[
        go.Bar(x=list(object_counts.keys()), y=list(object_counts.values()), marker=dict(color=["blue", "red", "green"]))
    ])
    fig.update_layout(title="Object Detection Count", xaxis_title="Objects", yaxis_title="Count", yaxis=dict(range=[0, max(object_counts.values(), default=1) + 2]))

    return fig

# Function to run YOLO detection
def run_yolo():
    global object_counts
    detection_results = []
    while cap.isOpened():
        # Read the camara video as frames
        ret, frame = cap.read()
        if not ret:
            break

        # Run model using the frames as input
        results = model(frame)

        # Reset object count per frame
        object_counts = {"pizza": 0, "burger": 0, "fries": 0}

        for result in results:
            for box in result.boxes:
                # Get class ID (0 = burger, 1 = pizza, 2 = fries)
                cls_id = int(box.cls[0])  
                # Get class name
                class_name = model.names[cls_id]  
                
                # Count objects (only pizza & burger)
                if class_name in object_counts:
                    object_counts[class_name] += 1

                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])  
                # Draw bounding box in the camara view
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # name the object
                cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Save detection resultseach second

        #if time.strftime("%S") % 2 == 0:
        detection_results.append({
            "Time": time.strftime("%Y-%m-%d %H:%M:%S"),
            "burger": object_counts["burger"],
            "pizza": object_counts["pizza"],
            "fries": object_counts["fries"]
        })

        # Display object count on screen
        y_offset = 20
        for class_name, count in object_counts.items():
            text = f"{class_name}: {count}"
            cv2.putText(frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            y_offset += 30

        # Show video feed
        cv2.imshow("YOLOv8 Object Detection", frame)



        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            with open("detection_results.txt", "w") as f:
                json.dump(detection_results, f, indent=4)
            break

    cap.release()
    cv2.destroyAllWindows()

# Run YOLO in a separate thread
threading.Thread(target=run_yolo, daemon=True).start()

# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)



0: 480x640 (no detections), 70.3ms
Speed: 4.0ms preprocess, 70.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 76.8ms
Speed: 2.7ms preprocess, 76.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 67.7ms
Speed: 1.9ms preprocess, 67.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 78.8ms
Speed: 2.2ms preprocess, 78.8ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 74.2ms
Speed: 1.6ms preprocess, 74.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 97.3ms
Speed: 2.2ms preprocess, 97.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 61.5ms
Speed: 1.4ms preprocess, 61.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 50.4ms
Speed: 1.6ms preprocess, 50.4ms i